# Статистичні методи
Індивідуальне завдання 6

Токарєв Павло, МФ-51

## Вхідні дані

In [55]:
from functools import reduce
from random import seed
TEXT1= open("data/ukrainian/1.txt").read()
TEXT2 = open("data/ukrainian/2.txt").read()
TEXT3 = open("data/ukrainian/3.txt").read()
TEXT4 = open("data/ukrainian/4.txt").read()
TEXT5 = open("data/ukrainian/5.txt").read()
TEXT6 = open("data/ukrainian/6.txt").read()

TEXTS = [TEXT1, TEXT2, TEXT3, TEXT4, TEXT5, TEXT6]

## Довжини текстів

In [56]:
[len(text) for text in TEXTS]

[7283, 11477, 8447, 8988, 6283, 7830]

## Вилучення непотрібних символів

In [57]:
import re
exclude = r"[0123456789/%²°+()]"
replace = r"[;:?.!,«»\n“”]"

def process(text: str) -> str:
    text = re.compile(exclude).sub("", text)
    text = re.compile(replace).sub(" ", text)
    text = text.lower()
    return text

TEXTS = [process(text) for text in TEXTS]

## Наявні символи

In [58]:
import itertools
symbols = set(itertools.chain.from_iterable(TEXTS))
symbols

{' ',
 '"',
 "'",
 '-',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'v',
 'x',
 'y',
 'z',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'ж',
 'з',
 'и',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'у',
 'ф',
 'х',
 'ц',
 'ч',
 'ш',
 'щ',
 'ь',
 'ю',
 'я',
 'є',
 'і',
 'ї',
 'ґ',
 '–',
 '—',
 '’'}

## Токенізація

In [59]:
from typing import List

def tokenize(text: str) -> List[str]:
    return [m.group(0) for m in re.compile(r"[\w’']+").finditer(text)]

tokens = [tokenize(text) for text in TEXTS]
tokens[:30]

[['бюджетна',
  'криза',
  'чому',
  'уряд',
  'заблокував',
  'видатки',
  'і',
  'до',
  'чого',
  'це',
  'може',
  'привести',
  'кабмін',
  'зіткнувся',
  'з',
  'бюджетними',
  'проблемами',
  'і',
  'тепер',
  'вимушений',
  'притримати',
  'видатки',
  'що',
  'це',
  'означає',
  'скільки',
  'коштів',
  'не',
  'вистачає',
  'кого',
  'це',
  'стосується',
  'та',
  'які',
  'є',
  'ризики',
  'олексій',
  'гончарук',
  'доручив',
  'міністру',
  'фінансів',
  'оксані',
  'маркаровій',
  'не',
  'проводити',
  'низку',
  'видатків',
  'до',
  'кінця',
  'року',
  'причина',
  'потрібно',
  'вкластися',
  'в',
  'запланований',
  'розмір',
  'бюджетного',
  'дефіциту',
  'це',
  'вже',
  'вдруге',
  'за',
  'останні',
  'пів',
  'року',
  'мінфін',
  'призупиняє',
  'проведення',
  'певної',
  'частини',
  'видатків',
  'які',
  'видатки',
  'не',
  'будуть',
  'профінансовані',
  'наскільки',
  'небезпечний',
  'цей',
  'тренд',
  'і',
  'з',
  'чим',
  'він',
  "пов'язаний",

In [60]:
[len(set(text_tokens)) for text_tokens in tokens]

[575, 843, 679, 848, 548, 607]

In [61]:
len(set(itertools.chain.from_iterable(tokens)))

3156

In [62]:
## Побудова матриці частот

In [63]:
import numpy as np

def build_table(tokens: List[List[str]]) -> (List[str], np.array):
    lexicon = list(set(itertools.chain.from_iterable(tokens)))
    n = len(lexicon)
    k = len(tokens)
    
    index = {w: i for i, w in enumerate(lexicon)}
    matrix = np.zeros((n,k))
    for i, token_set in enumerate(tokens):
        for word in token_set:
            matrix[index[word]][i]+=1
    return lexicon, matrix

lexicon, matrix = build_table(tokens)
matrix

array([[0., 0., 3., 0., 0., 0.],
       [0., 3., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0.]])

## Нормалізація $\sqrt{x}$

In [64]:
matrix = np.sqrt(matrix)
matrix

array([[0.        , 0.        , 1.73205081, 0.        , 0.        ,
        0.        ],
       [0.        , 1.73205081, 0.        , 0.        , 0.        ,
        0.        ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        ]])

## Нормалізація за допомогою зворотнії частот

In [65]:
def normalize(m : np.array) -> np.array:
    a = np.zeros(m.shape)
    n, k = m.shape
    for i in range(n):
        ni = 0
        for j in range(k):
            if m[i][j] > 0:
                ni +=1
        id = np.log(k/ni)
        for j in range(k):
            a[i][j] = m[i][j] * id
    return a
matrix = normalize(matrix)
matrix

array([[0.        , 0.        , 3.10341844, 0.        , 0.        ,
        0.        ],
       [0.        , 3.10341844, 0.        , 0.        , 0.        ,
        0.        ],
       [1.79175947, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 1.79175947, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.79175947],
       [0.        , 0.        , 1.79175947, 0.        , 0.        ,
        0.        ]])

## Видалення спільних термів

In [66]:
def remove_common(a: np.array) -> np.array:
    return a[np.prod(a, axis=1) == 0]

matrix = remove_common(matrix)
matrix

array([[0.        , 0.        , 3.10341844, 0.        , 0.        ,
        0.        ],
       [0.        , 3.10341844, 0.        , 0.        , 0.        ,
        0.        ],
       [1.79175947, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 1.79175947, 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.79175947],
       [0.        , 0.        , 1.79175947, 0.        , 0.        ,
        0.        ]])

## Назви статей
0. Бюджетна криза: чому уряд заблокував видатки і до чого це може привести
1. Округи та префекти. Як в Україні можуть з'явитися "смотрящіє" від Зеленського
2. Максим Поляков, Firefly: Космічний бізнес — це бізнес політичний і йому потрібна державна підтримка
3. Українські "Схованки": чим бере справді крутий перший наш серіал
4. Боротьба за вибори: що покладе край протестам у Грузії
5. Другий референдум за Brexit: які наслідки матиме історична перемога Бориса Джонсона

## Обчислення $cos$ відстаней

In [67]:
def calc_distances(matrix: np.array) -> np.array:
    n, k = matrix.shape
    d = np.zeros((k,k))
    for i in range(k):
        for j in range(k):
            a = matrix[:, i]
            b = matrix[:, j]
            d[i][j] = np.dot(a.reshape((1,n)), b.reshape((n,1)))/\
                      (np.sqrt(np.sum(np.power(a, 2)))*np.sqrt(np.sum(np.power(b, 2))))
    return d

distances = calc_distances(matrix)
distances

array([[1.        , 0.02602841, 0.03075456, 0.01303655, 0.01942801,
        0.02636176],
       [0.02602841, 1.        , 0.04490818, 0.02916655, 0.0466422 ,
        0.04162933],
       [0.03075456, 0.04490818, 1.        , 0.02616161, 0.01731084,
        0.02190851],
       [0.01303655, 0.02916655, 0.02616161, 1.        , 0.01967823,
        0.02112593],
       [0.01942801, 0.0466422 , 0.01731084, 0.01967823, 1.        ,
        0.05111553],
       [0.02636176, 0.04162933, 0.02190851, 0.02112593, 0.05111553,
        1.        ]])

## Обчислення $Jaccard$ відстаней

In [68]:
def calc_jaccard() -> np.array:
    k = len(tokens)
    sets = list(map(set, tokens))
    m = np.zeros((k,k))
    for i, t1 in enumerate(sets):
        for j, t2 in enumerate(sets):
            if i == j:
                res = 1
            else:
                res = (len(t1.intersection(t2))/len(t1.union(t2)))
            m[i][j] = res
    return m
jaccard = calc_jaccard()
jaccard

array([[1.        , 0.07996954, 0.08477509, 0.0611484 , 0.08188825,
        0.09242144],
       [0.07996954, 1.        , 0.10289855, 0.08466966, 0.10748408,
        0.11452729],
       [0.08477509, 0.10289855, 1.        , 0.07915194, 0.08488064,
        0.08983051],
       [0.0611484 , 0.08466966, 0.07915194, 1.        , 0.07384615,
        0.08098068],
       [0.08188825, 0.10748408, 0.08488064, 0.07384615, 1.        ,
        0.13346418],
       [0.09242144, 0.11452729, 0.08983051, 0.08098068, 0.13346418,
        1.        ]])